In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime,timedelta
import numpy as np

def competenum(kkwwn):
    try:
        kwuu = 'https://search.naver.com/search.naver?where=influencer&sm=tab_jum&query={}'.format(kkwwn)
        kwres = requests.get(kwuu)
        kwpage = BeautifulSoup(kwres.text,'html.parser')
        compnum = re.search(r'\d+',kwpage.find('span',{'class':'num_bx _creator_counts'}).text.replace('\n','')).group()
        return compnum
    except:
        kwuu = 'https://search.naver.com/search.naver?query={}&sm=tab_nmr&where=influencer'.format(kkwwn)
        kwres = requests.get(kwuu)
        kwpage = BeautifulSoup(kwres.text,'html.parser')
#         compnum = re.search(r'\d+',kwpage.find('span',{'class':'num_bx _creator_counts'}).text.replace('\n','')).group()
        compnum = len(kwpage.find_all('div',{'class':'keyword_box_wrap api_ani_send'}))
        return compnum

now = datetime.now()
standard = now - timedelta(hours=24)
standard = datetime.strftime(standard, "%Y-%m-%d %H:%M")

In [2]:
def calculatecompete(influencerid):
    # ownerID 값찾기
    influid = influencerid
    print(influid)
    influu = 'https://in.naver.com/{}'.format(influid)
    res = requests.get(influu)
    pp = BeautifulSoup(res.text, 'html.parser')

    ownerId = re.search(r'"ownerId":\d+',pp.find('script',{'id':'initialState'}).text).group()
    ownerId = ownerId.split(':')[-1]
    print(ownerId)
    header ={
    'accept': '*/*',
    # 'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-length': '1680',
    'content-type': 'application/json',
#     'cookie': 'NNB=R2SJKPLCFBHGE; MM_NEW=1; NFS=2; _fbp=fb.1.1649375524994.1716697008; _gcl_au=1.1.421096904.1649375525; ASID=79a65bc3000001801b32df9b00000052; NaverSuggestUse=use%26unuse; _ga_N90K8EJMQ3=GS1.1.1651797479.1.1.1651797500.0; nx_ssl=2; _ga=GA1.2.234153349.1649375525; _gid=GA1.2.1117221281.1652349558; _ga_7VKFYR6RV1=GS1.1.1652362479.13.0.1652362479.60; _naver_usersession_=j4T2cOr2BB7NxRoQaaab/kO+',
    'inf-fe-version': '2022.03.22',
    'origin': 'https://in.naver.com',
    'referer': 'https://in.naver.com/{}/challenge?sortType=LAST_UPDATE'.format(influencerid),
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"10.0.0"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'
    }

    dd1 = {'operationName': 'getParticipatedKeywords',
     'variables': {'input': {'sort': 'LAST_UPDATE', 'ownerId': ownerId},
      'paging': {'limit': 20}},
     'query': 'query getParticipatedKeywords($input: ParticipatedKeywordsInput!, $paging: PagingInput!) {\n  participatedKeywords(input: $input, paging: $paging) {\n    items {\n      ...ParticipatedKeywordViewWithChallenges\n      __typename\n    }\n    paging {\n      nextCursor\n      total\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment ParticipatedKeywordViewWithChallenges on ParticipatedKeywordView {\n  ...ParticipatedKeywordView\n  ownerId\n  contents {\n    ...ChallengeContent\n    __typename\n  }\n  __typename\n}\n\nfragment ParticipatedKeywordView on ParticipatedKeywordView {\n  categoryId\n  id\n  lastChallengedAt\n  name\n  rank\n  viewCount\n  challengeCount\n  __typename\n}\n\nfragment ChallengeContent on ChallengeContent {\n  id\n  challengeId\n  originId\n  originGdid\n  url\n  mobileUrl\n  type\n  title\n  gifUrl\n  playTime\n  createdAt\n  viewCount\n  likeCount\n  mediaCount\n  description\n  thumbnailUrl\n  serviceType\n  twitterScreenName\n  authorName\n  authorProfileImageUrl\n  videoId\n  inKey\n  parentTweetAuthorName\n  parentTweetAuthorProfileImageUrl\n  parentTweetCreatedAt\n  parentTweetDescription\n  parentTweetThumbnailUrl\n  parentTweetTwitterScreenName\n  parentTweetUrl\n  isRetweetComment\n  urlLinkMap\n  thumbnails\n  clickCodeKey\n  salePrice\n  discountedSalePrice\n  discountedRatio\n  mobileDiscountedSalePrice\n  mobileDiscountedRatio\n  isNew\n  isBest\n  contentType\n  serviceId\n  participated\n  participatedAnotherKeyword\n  dimmed\n  isBlogMarketPost\n  __typename\n}\n'}

    url = 'https://in.naver.com/graphql'
    res = requests.post(url, headers=header, json = dd1)
    page = res.json()
    curs = page['data']['participatedKeywords']['paging']['nextCursor']
    contlst = []
    for contents in page['data']['participatedKeywords']['items']:
        dic = {}
        dic['키워드'] = contents['name']
        dic['랭크'] = contents['rank']
        dic['게시글url'] = contents['contents'][0]['url']
        dic['게시시간'] = contents['lastChallengedAt']
        contlst.append(dic)
    if curs != None:
        
        dd2 = {'operationName': 'getParticipatedKeywords',
         'variables': {'input': {'sort': 'LAST_UPDATE', 'ownerId': ownerId},
          'paging': {'limit': 20, 'cursor':curs}},
         'query': 'query getParticipatedKeywords($input: ParticipatedKeywordsInput!, $paging: PagingInput!) {\n  participatedKeywords(input: $input, paging: $paging) {\n    items {\n      ...ParticipatedKeywordViewWithChallenges\n      __typename\n    }\n    paging {\n      nextCursor\n      total\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment ParticipatedKeywordViewWithChallenges on ParticipatedKeywordView {\n  ...ParticipatedKeywordView\n  ownerId\n  contents {\n    ...ChallengeContent\n    __typename\n  }\n  __typename\n}\n\nfragment ParticipatedKeywordView on ParticipatedKeywordView {\n  categoryId\n  id\n  lastChallengedAt\n  name\n  rank\n  viewCount\n  challengeCount\n  __typename\n}\n\nfragment ChallengeContent on ChallengeContent {\n  id\n  challengeId\n  originId\n  originGdid\n  url\n  mobileUrl\n  type\n  title\n  gifUrl\n  playTime\n  createdAt\n  viewCount\n  likeCount\n  mediaCount\n  description\n  thumbnailUrl\n  serviceType\n  twitterScreenName\n  authorName\n  authorProfileImageUrl\n  videoId\n  inKey\n  parentTweetAuthorName\n  parentTweetAuthorProfileImageUrl\n  parentTweetCreatedAt\n  parentTweetDescription\n  parentTweetThumbnailUrl\n  parentTweetTwitterScreenName\n  parentTweetUrl\n  isRetweetComment\n  urlLinkMap\n  thumbnails\n  clickCodeKey\n  salePrice\n  discountedSalePrice\n  discountedRatio\n  mobileDiscountedSalePrice\n  mobileDiscountedRatio\n  isNew\n  isBest\n  contentType\n  serviceId\n  participated\n  participatedAnotherKeyword\n  dimmed\n  isBlogMarketPost\n  __typename\n}\n'}

        res2 = requests.post(url, headers=header, json = dd2)
        page2 = res2.json()
        curs = page2['data']['participatedKeywords']['paging']['nextCursor']
        for contents in page2['data']['participatedKeywords']['items']:
            dic = {}
            dic['키워드'] = contents['name']
            dic['랭크'] = contents['rank']
            dic['게시글url'] = contents['contents'][0]['url']
            dic['게시시간'] = contents['lastChallengedAt']
            contlst.append(dic)

    contdf = pd.DataFrame(contlst)
    if len(contdf) == 0:
         return ('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')
    else:
        contdf['게시시간'] = contdf['게시시간'].apply(lambda x: ' '.join(str(x).split('T'))[:16])
        contdf = contdf.loc[contdf['게시시간'] < standard]
        contdf.reset_index(inplace=True, drop=True)



        if len(contdf) < 10 and curs != None:

            for i in range(5):
                dd2 = {'operationName': 'getParticipatedKeywords',
                 'variables': {'input': {'sort': 'LAST_UPDATE', 'ownerId': ownerId},
                  'paging': {'limit': 20, 'cursor':curs}},
                 'query': 'query getParticipatedKeywords($input: ParticipatedKeywordsInput!, $paging: PagingInput!) {\n  participatedKeywords(input: $input, paging: $paging) {\n    items {\n      ...ParticipatedKeywordViewWithChallenges\n      __typename\n    }\n    paging {\n      nextCursor\n      total\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment ParticipatedKeywordViewWithChallenges on ParticipatedKeywordView {\n  ...ParticipatedKeywordView\n  ownerId\n  contents {\n    ...ChallengeContent\n    __typename\n  }\n  __typename\n}\n\nfragment ParticipatedKeywordView on ParticipatedKeywordView {\n  categoryId\n  id\n  lastChallengedAt\n  name\n  rank\n  viewCount\n  challengeCount\n  __typename\n}\n\nfragment ChallengeContent on ChallengeContent {\n  id\n  challengeId\n  originId\n  originGdid\n  url\n  mobileUrl\n  type\n  title\n  gifUrl\n  playTime\n  createdAt\n  viewCount\n  likeCount\n  mediaCount\n  description\n  thumbnailUrl\n  serviceType\n  twitterScreenName\n  authorName\n  authorProfileImageUrl\n  videoId\n  inKey\n  parentTweetAuthorName\n  parentTweetAuthorProfileImageUrl\n  parentTweetCreatedAt\n  parentTweetDescription\n  parentTweetThumbnailUrl\n  parentTweetTwitterScreenName\n  parentTweetUrl\n  isRetweetComment\n  urlLinkMap\n  thumbnails\n  clickCodeKey\n  salePrice\n  discountedSalePrice\n  discountedRatio\n  mobileDiscountedSalePrice\n  mobileDiscountedRatio\n  isNew\n  isBest\n  contentType\n  serviceId\n  participated\n  participatedAnotherKeyword\n  dimmed\n  isBlogMarketPost\n  __typename\n}\n'}

                res2 = requests.post(url, headers=header, json = dd2)
                page2 = res2.json()
                for contents in page2['data']['participatedKeywords']['items']:
                    dic = {}
                    dic['키워드'] = contents['name']
                    dic['랭크'] = contents['rank']
                    dic['게시글url'] = contents['contents'][0]['url']
                    dic['게시시간'] = contents['lastChallengedAt']
                    contlst.append(dic)
                curs = page2['data']['participatedKeywords']['paging']['nextCursor']
            contdf = pd.DataFrame(contlst)
            contdf['게시시간'] = contdf['게시시간'].apply(lambda x: ' '.join(str(x).split('T'))[:16])
            contdf = contdf.loc[contdf['게시시간'] < standard]
            contdf.reset_index(inplace=True, drop=True)
            if len(contdf) < 10:
                kwcnt = ''
                postcnt = ''
                competeavr = ''
                mincompet = ''
                maxcompet = ''
                return (kwcnt, postcnt, competeavr, mincompet, maxcompet)



        condf2 = contdf[:10]
        if condf2.게시글url.nunique() >= 3:
            kwcnt = 10
            postcnt = condf2.게시글url.nunique()
            ddf = condf2
        else:
            postlst = list(contdf.게시글url.unique())[:3]
            ddf = contdf.loc[contdf['게시글url'].isin(postlst)]
            kwcnt = ddf.키워드.nunique()
            postcnt = 3
        ddf.reset_index(inplace=True, drop=True)
        print(kwcnt, postcnt)
        ddf['키워드 경쟁자수'] = ddf['키워드'].apply(lambda x: competenum(x))
        ddf['키워드 경쟁자수'] = ddf['키워드 경쟁자수'].apply(lambda x: int(x))
        ddf['키워드 경쟁률'] = ddf['랭크'] / ddf['키워드 경쟁자수']
        if len(ddf) != 0:
            competeavr = np.mean(ddf['키워드 경쟁률'])
            mincompet = min(ddf['키워드 경쟁률'])
            maxcompet = max(ddf['키워드 경쟁률'])
        

            return (kwcnt, postcnt, competeavr, mincompet, maxcompet)
        else:
            return ('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

In [4]:
df = pd.read_excel("인플루언서 크롤링/인플루언서 크롤링_뷰티_인플루언서리스트_220718.xlsx")
df['키워드 경쟁강도 데이터'] = ''
df

,Unnamed: 0,ID,인플루언서명,설명,팬수,블로그URL,naverID,이메일,키워드 경쟁강도 데이터
0,0,motomakeup,모또 메이크업,메이크업 아티스트,11074,https://blog.naver.com/makeup_moto,makeup_moto,makeup_moto@naver.com,
1,1,mooney1987,미미꽃,뷰티 전문블로거,17481,https://blog.naver.com/mooney1987,mooney1987,mooney1987@naver.com,
2,2,zzuneduck,쭌이덕,뷰티 전문블로거,17742,https://blog.naver.com/rayharp,rayharp,rayharp@naver.com,
3,3,bora,뷰스타 쪼,뷰티 전문블로거,9720,https://blog.naver.com/bora871217,bora871217,bora871217@naver.com,
4,4,taeeun,이람님,뷰티 전문블로거,8156,https://blog.naver.com/qmfosej,qmfosej,qmfosej@naver.com,
...,...,...,...,...,...,...,...,...,...
1997,1997,whozmakeup,후메이크업,뷰티 전문블로거,17,정보없음,정보없음,정보없음,
1998,1998,gayoon,가윤,뷰티 전문블로거,17,정보없음,정보없음,정보없음,
1999,1999,misssohn,MISSSOHN,뷰티 전문블로거,15,정보없음,정보없음,정보없음,
2000,2000,chahongroom_yeeun,조예은,뷰티 전문블로거,14,정보없음,정보없음,정보없음,


In [8]:
for i in range(1685, len(df)):
    ifid = df.loc[i,'ID']
    print()
    print(i)
    df.at[i,'키워드 경쟁강도 데이터'] = calculatecompete(ifid)
    print(df.loc[i,'키워드 경쟁강도 데이터'])
    
df['키워드 수'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[0])
df['포스팅 수'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[1])
df['평균 키워드 경쟁강도'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[2])
df['최소 키워드 경쟁강도'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[3])
df['최대 키워드 경쟁강도'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[4])
df = df[['ID', '인플루언서명', '설명', '팬수', '블로그URL', 'naverID', '이메일', '키워드 수', '포스팅 수', '평균 키워드 경쟁강도', '최소 키워드 경쟁강도', '최대 키워드 경쟁강도']]
df


1685
lily_beauty
118337807647904
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1686
enesalon
125000466250144
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1687
sobeaute
147217144972320
2 3
(2, 3, 0.8171921921921922, 0.8055555555555556, 0.8288288288288288)

1688
gasenglish
125391745034688
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1689
bymalla
119762473056256
2 3
(2, 3, 0.7742222598965839, 0.5697211155378487, 0.9787234042553191)

1690
huruluk_mukstagram
131890484653152
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1691
vkeo3136
210968698333344
10 8


C:\Users\onrik\AppData\Local\Temp/ipykernel_18232/2067184330.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['키워드 경쟁자수'] = ddf['키워드'].apply(lambda x: competenum(x))
C:\Users\onrik\AppData\Local\Temp/ipykernel_18232/2067184330.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['키워드 경쟁자수'] = ddf['키워드 경쟁자수'].apply(lambda x: int(x))
C:\Users\onrik\AppData\Local\Temp/ipykernel_18232/2067184330.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

(10, 8, 0.4986323265073471, 0.2708333333333333, 0.9265536723163842)

1692
jhair_ujin
150541416724352
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1693
mute0812
131846047243328
10 5
(10, 5, 0.9256187492255592, 0.841025641025641, 1.015267175572519)

1694
h_shoo
115871966839520
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1695
leesunminnn
117249806865696
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1696
hwane_online
117275297195232
2 3
(2, 3, 1.0804020100502512, 1.0804020100502512, 1.0804020100502512)

1697
jina9
168476982517536
2 3
(2, 3, 1.0944366526414213, 0.8985507246376812, 1.2903225806451613)

1698
dudwn4229
131841482316864
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1699
youwho
120093631187008
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1700
dolidooly25
118321777386816
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1701
keykey5397
117232584364928
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1702
pigyoni
163003596533088
('챌린지 없음', '챌린

218433071775200
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1789
jobi_beauty
118231017366784
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1790
artist_jisu
162814340484448
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1791
qnkk
151591514748992
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1792
o.reon
128322078465024
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1793
lux.som
117284286760160
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1794
doaa
153110504663104
2 3
(2, 3, 0.7544273677640818, 0.5948275862068966, 0.9140271493212669)

1795
daily_jiji
121066650142752
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1796
boramm9004
144467475645088
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1797
aluu_p_mk_p
125133564762560
1 3
(1, 3, 0.8784530386740331, 0.8784530386740331, 0.8784530386740331)

1798
yocookie
117562594383136
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1799
binnalyoon
139839296949728
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음

190115724443488
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1887
euneeew
147664476116032
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1888
shunii
117279690310816
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1889
yell_channel
115906234614400
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1890
iy91
116120875627136
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1891
ming_u.beauty
153430919920032
10 8
(10, 8, 0.8909575716792042, 0.5641025641025641, 1.0644391408114557)

1892
geon_u
162083277176064
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1893
seno
133161479433376
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1894
beautifymeeh
129654774013024
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1895
im-jiwoo
144195258356416
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1896
goeun
156087153957536
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1897
lydii_hellolyds
117308070450400
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1898

128703429721056
1 3
(1, 3, 0.9147286821705426, 0.9147286821705426, 0.9147286821705426)

1985
saena7777
153812887644608
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1986
chaeni.gram
117908353557792
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1987
hello2691
145037764928192
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1988
minhwa_o
119682255841248
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1989
dd9987_
123639856740608
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1990
withshyan
164013425003904
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1991
leedsning
141582635949760
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1992
stellalala_u
135286926707840
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1993
oh_dya
132461277584480
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1994
kang_woo
144787140354656
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

1995
iamyelim
179188643875520
('챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음', '챌린지 없음')

19

,ID,인플루언서명,설명,팬수,블로그URL,naverID,이메일,키워드 수,포스팅 수,평균 키워드 경쟁강도,최소 키워드 경쟁강도,최대 키워드 경쟁강도
0,motomakeup,모또 메이크업,메이크업 아티스트,11074,https://blog.naver.com/makeup_moto,makeup_moto,makeup_moto@naver.com,10,4,0.043809,0.003378,0.199081
1,mooney1987,미미꽃,뷰티 전문블로거,17481,https://blog.naver.com/mooney1987,mooney1987,mooney1987@naver.com,10,3,0.007842,0.001603,0.030488
2,zzuneduck,쭌이덕,뷰티 전문블로거,17742,https://blog.naver.com/rayharp,rayharp,rayharp@naver.com,10,3,0.068996,0.007937,0.222222
3,bora,뷰스타 쪼,뷰티 전문블로거,9720,https://blog.naver.com/bora871217,bora871217,bora871217@naver.com,10,5,0.02024,0.002506,0.083333
4,taeeun,이람님,뷰티 전문블로거,8156,https://blog.naver.com/qmfosej,qmfosej,qmfosej@naver.com,10,3,0.059761,0.007194,0.123457
...,...,...,...,...,...,...,...,...,...,...,...,...
1997,whozmakeup,후메이크업,뷰티 전문블로거,17,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음
1998,gayoon,가윤,뷰티 전문블로거,17,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음
1999,misssohn,MISSSOHN,뷰티 전문블로거,15,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음
2000,chahongroom_yeeun,조예은,뷰티 전문블로거,14,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음


In [9]:
df = df[['ID', '인플루언서명', '설명', '팬수', '블로그URL', 'naverID', '이메일', '키워드 수', '포스팅 수', '평균 키워드 경쟁강도', '최소 키워드 경쟁강도', '최대 키워드 경쟁강도']]
df.to_excel("뷰티_인플루언서_경쟁강도.xlsx", index=False)

### 중간 저장해서 불러왔을 때!

In [67]:
df = pd.read_excel('생활건강_인플루언서_경쟁강도_220608_1차.xlsx')
for didi in range(len(df)):
    print(didi)
    datalst = df.loc[didi,'키워드 경쟁강도 데이터']
    try:
        df.loc[didi,'키워드 경쟁강도 데이터'] = ast.literal_eval(datalst)
    except:
        datalst = datalst.replace('inf',"'없음'")
        datalst = datalst.replace('nan',"'없음'")
        df.loc[didi,'키워드 경쟁강도 데이터'] = ast.literal_eval(datalst)
df['키워드 수'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[0])
df['포스팅 수'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[1])
df['평균 키워드 경쟁강도'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[2])
df['최소 키워드 경쟁강도'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[3])
df['최대 키워드 경쟁강도'] = df['키워드 경쟁강도 데이터'].apply(lambda x: x[4])
df = df[['ID', '인플루언서명', '설명', '팬수', '블로그URL', 'naverID', '이메일', '키워드 수', '포스팅 수', '평균 키워드 경쟁강도', '최소 키워드 경쟁강도', '최대 키워드 경쟁강도']]
df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,ID,인플루언서명,설명,팬수,블로그URL,naverID,이메일,키워드 수,포스팅 수,평균 키워드 경쟁강도,최소 키워드 경쟁강도,최대 키워드 경쟁강도
0,binaida,셀럽주부,생활건강 매거진 에디터,팬 3.6만+,https://blog.naver.com/zzzbina,zzzbina,zzzbina@naver.com,10,4,0.218679,0.012821,0.714286
1,ryuae4,류애,생활건강 전문블로거,팬 2.1만+,https://blog.naver.com/ryuae4,ryuae4,ryuae4@naver.com,10,3,0.064291,0.026316,0.2
2,barun,바른사람,의학 전문가,팬 2.7만+,정보없음,정보없음,정보없음,10,4,0.051544,0.02439,0.095238
3,egofkhk,김센스,심리상담 전문가,팬 2만+,https://blog.naver.com/egofkhk,egofkhk,egofkhk@naver.com,10,7,0.24894,0.033333,0.866667
4,livinghealth365,언니의건강,영양 전문가,팬 2.9만+,https://blog.naver.com/goodjoyful,goodjoyful,goodjoyful@naver.com,10,5,0.072303,0.013889,0.308824
...,...,...,...,...,...,...,...,...,...,...,...,...
436,hj.seo_othilia,hj.seo,생활건강 전문블로거,팬 85,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음
437,yakdi2080,알쓸신약,약학 전문가,팬 120,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음
438,thej,THE J,테라피스트,팬 86,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음
439,jupanda,쥬팬더,의학 전문가,팬 64,정보없음,정보없음,정보없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음,챌린지 없음


In [68]:
df.to_excel("220609_생활건강_인플루언서 경쟁강도.xlsx", index=False)

## 인플루언서 경쟁강도 트래킹

### 효과: 1. SEO진행 인플루언서 수 감소, 
### &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  2. 인플루언서 수 감소로 인한 SEO 비용 감소
### &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  3. 상위노출 효과 상승

### 기존 SEO 방식 & 경쟁강도 이용 방식 공통점:
- 키워드 선정:
    - 키워드 검색 결과에서 인플루언서 TAB or VIEW TAB 위치 확인(해당 탭들이 상위노출)
- 내용(정성적):
    - 특정 키워드에서 상위노출된 포스트 정성 분석 (썸네일, 포함 키워드 수 등) 후 벤치마킹
    - 내용에는 링크 삽입 x (링크 삽입할 경우, 광고라는 특성이 강조되기 때문에 부정적 영향을 끼칠 수 있다고 판단)

### 기존 SEO 방식 & 경쟁강도 이용 방식 차이점:

- 기존 SEO 방식 인플루언서 선정:
    - 특정 카테고리의 인플루언서 리스트 추출
    - 모든 인플루언서 컨택 후 회신 온 인플루언서 중 직접 선정
    
- 경쟁강도 이용 방식 인플루언서 선정:
    - 특정 카테고리의 인플루언서 리스트 추출
    - 모든 인플루언서들의 경쟁강도 확인
    - 진행하려고 하는 키워드에서 상위노출된 인플루언서의 최소 경쟁강도를 기준으로 기준 이상 인플루언서 추출
    - 추출된 인플루언서 중 선정 후 컨택
        - 왜 모든 인원을 진행하지 않냐면, SEO는 길게 가져가야할 전략으로 초반에 모든 패를 사용하지 않고 뽑아씀
- 경쟁강도 이용 방식의 장점:
    - 기존 방식보다 진행하는 인플루언서의 수가 적기 때문에 각 인플루언서마다의 경쟁강도에 따라 비용 차등 적용 가능
    - 상위노출을 기반으로 계산한 경쟁강도이기 때문에 상위노출 효과 상승